Required Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import json

from sklearn.model_selection import train_test_split

Initializing variables and preparing data

In [2]:
SERVER_IP = "127.0.0.1"
pp_PORT = 5000
eval_PORT = 5002

col_names = ["VB","smcAC","smcDC","vib_table","vib_spindle","AE_table","AE_spindle"]
target = "VB"
df = pd.read_csv(r"C:\Users\chh-st\Documents\GitHub\fabos-base-service-ai-pipeline\06_Test_Framework\tests\tests\WZ_Verschleiss.csv",
                header = None,
                names = col_names,
                skiprows=1,
                index_col= False,									
                sep = ',')

model_name = "use_case_test_model"
test_size = 0.7

print(df.head(5))

    VB     smcAC     smcDC  vib_table  vib_spindle  AE_table  AE_spindle
0  0.0 -0.017090  0.625000   0.078125     0.314941  0.087280    0.103760
1  0.0  0.263672  0.810547   0.085449     0.301514  0.098267    0.123291
2  0.0  0.207520  0.781250   0.078125     0.303955  0.092163    0.104980
3  0.0  0.302734  0.849609   0.073242     0.300293  0.095215    0.111084
4  0.0  0.239258  1.098633   0.083008     0.299072  0.083008    0.092163


Pre-Processing data to get the data which was used to train the model

In [4]:
# Which columns to process
col = ["smcAC", "smcDC", "vib_table", "vib_spindle", "AE_table", "AE_spindle"]

# Set parameters
my_param = {"col": col}

### DATA IN PANDAS DF ###
files = {
    'data': ("data", df.to_json(), 'application/json')
}
r = requests.post('http://{0}:{1}/nan'.format(SERVER_IP, pp_PORT), params=my_param, files=files)
df_json = r.json()
resp_df = pd.DataFrame.from_dict(df_json)
# print(resp_df.head(1))
        
# Call preprocessing service - standardize values in columns "col"
files = {
    'data': ("data", resp_df.to_json(), 'application/json')
}
r = requests.post('http://{0}:{1}/std'.format(SERVER_IP, pp_PORT), params=my_param, files=files)
df_json = r.json()
resp_df = pd.DataFrame.from_dict(df_json)
# print(resp_df.head(1))

# Call preprocessing service - Remove strongly correlating features
files = {
    'data': ("data", resp_df.to_json(), 'application/json')
}        
r = requests.post('http://{0}:{1}/rm_corr'.format(SERVER_IP, pp_PORT), params=my_param, files=files)
df_json = r.json()
resp_df = pd.DataFrame.from_dict(df_json)
print(resp_df.head(5))

    VB     smcAC  vib_spindle  AE_spindle
0  0.0  0.061127    -1.042969   -1.518555
1  0.0  0.182129    -1.161133   -1.369141
2  0.0  0.157959    -1.139648   -1.508789
3  0.0  0.198975    -1.171875   -1.461914
4  0.0  0.171631    -1.182617   -1.607422


Evaluation service - single model evaluation

In [ ]:
tmp_df = resp_df
y = tmp_df[target].values
X = tmp_df.drop(target, axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state = 42) 

X_test_lists = X_test.tolist()
y_test_lists = y_test.tolist()

X_test_json_str = json.dumps(X_test_lists)
y_test_json_str = json.dumps(y_test_lists)

files = {
    'X_test': ("X_test", X_test_json_str, 'application/json'),
    'y_test': ("y_test", y_test_json_str, 'application/json'),
    'model': (model_name, open("C:/Users/chh-st/Documents/GitHub/fabos-base-service-ai-pipeline/" + model_name + '.zip', 'rb'), 'application/octet-stream'),
    'model_2': ("use_case_test_model_2", open("C:/Users/chh-st/Documents/GitHub/fabos-base-service-ai-pipeline/use_case_test_model_2.zip", 'rb'), 'application/octet-stream')
}

# Set parameters
metrics = ["mse", "mae", "mape"]
my_param = {"metric": metrics}

r = requests.post('http://{0}:{1}/evaluate'.format(SERVER_IP, eval_PORT), params=my_param, files=files)
resp = r.json()
print(resp)

r = requests.post('http://{0}:{1}/benchmark'.format(SERVER_IP, eval_PORT), params=my_param, files=files)
resp = r.json()
print(resp)